In [1]:
import numpy as np
from dataclasses import dataclass, field
from typing import *

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *
from sdf_world.network import *

import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3
import jax_dataclasses as jdc
from copy import deepcopy
from sdf_world.opt_builder import *

In [2]:
class ProblemBuilder:
    #constr_types = ["kinematics", "grasp_prob", "robot_dist"]
    def __init__(self, robots, home_configs, num_via_points=0, is_relaxed=True):
        self.robots = robots
        self.home_configs = deepcopy(home_configs)
        self.is_relaxed = is_relaxed
        SubProblem.robots = robots
        SubProblem.home_configs = self.home_configs
        #SubProblem.is_relaxed = is_relaxed
        SubProblem.num_viapoints = 0
        if not is_relaxed:
            SubProblem.num_viapoints = num_via_points

        self.action_seq: List[Action] = []
        self.keyframes: List[Keyframe] = []
        self.variables: List[Instance] = []
        self.parameters: List[Instance] = []
        self.constraints: List[Constraint] = []
        self.dim:int = 0
        self.cdim:int = 0
        self.fn_dict = {}
        # self.num_via_points = num_via_points
    
    @property
    def vdim(self):
        return len(np.hstack([var.coord for var in self.variables]))
    
    @property
    def xlb(self):
        return np.hstack([var.lb for var in self.variables])
    @property
    def xub(self):
        return np.hstack([var.ub for var in self.variables])
    
    
    @property
    def ordered_constr_list(self):
        return list(item for sublist in self.constr_dict.values() for item in sublist)
    
    @property
    def clb(self):
        return np.hstack([constr.lb for constr in self.ordered_constr_list])
    @property
    def cub(self):
        return np.hstack([constr.ub for constr in self.ordered_constr_list])
    
    
    @property
    def instances(self)-> List[Instance]:
        return self.variables + self.parameters

    @property
    def subproblems(self):
        return self.action_seq + self.keyframes

    @property
    def constr_dict(self)->Dict[str,List[Constraint]]:
        constr_dict = {constr_type:[] for constr_type in self.fn_dict}
        for constr in self.constraints:
            if constr.constr_type in constr_dict:
                constr_dict[constr.constr_type].append(constr)
        return constr_dict
    
    def build_problem(self, action_seq:List[Action], init:Keyframe, goal:Keyframe):
        self.parse_keyframes(action_seq, init, goal)
        self.set_waypoint_instances()
        self.set_planning_instances()
        self.set_constraints()

    def parse_keyframes(
            self, action_seq:List[Action], init:Keyframe, goal:Keyframe
        ):
        self.action_seq = deepcopy(action_seq)
        self.init = deepcopy(init)
        self.goal = deepcopy(goal)
        for i, action in enumerate(self.action_seq):
            action.idx = i
            action.set_instances()
        
        keyframes: List[Keyframe] = [self.init]
        for idx in range(len(self.action_seq)-1):
            curr, next = self.action_seq[idx], self.action_seq[idx+1]
            idx = len(keyframes)
            if isinstance(curr, MoveHold) and isinstance(next, MoveHold):
                keyframes.append(
                    Handover(idx, curr.robot, next.robot, curr.grasp, next.grasp))
            else:
                if isinstance(curr, MoveFree): #pick
                    placing_frame = self.init
                    picking_frame = next
                else: #place
                    placing_frame = self.goal
                    picking_frame = curr

                if self.is_relaxed:
                    keyframes.append(
                        RelaxedPickPlace(idx, curr.robot, 
                         picking_frame.grasp, placing_frame.obj_pose))
                else:
                    keyframes.append(
                        PickPlace(idx, curr.robot, 
                         picking_frame.grasp, placing_frame.obj_pose))
                
        keyframes.append(self.goal)
        for keyframe in keyframes:
            if isinstance(keyframe, Home):
                keyframe.set_instances(self.is_relaxed)    
            else: 
                keyframe.set_instances()

        self.keyframes = keyframes
        
        print("set action_seq and keyframes")

    def set_waypoint_instances(self):
        #set trajectory
        for i, action in enumerate(self.action_seq):
            action.idx = i
            prev_keyframe, next_keyframe = self.keyframes[i], self.keyframes[i+1]
            action.set_waypoints(prev_keyframe, next_keyframe)

    def set_planning_instances(self):
        instances = {}
        for p in self.subproblems:
            for instance in p.instances:
                instances[instance.name] = instance
        self.variables = [instance for instance in instances.values() if instance.is_var]
        self.parameters = [instance for instance in instances.values() if not instance.is_var]

        print("set variables and parameters")
    
    def set_constraints(self):
        for p in self.subproblems:
            p.set_constraints()
            self.constraints.extend(p.constraints)
        
        print("set constraints")

    def set_coordinates(self):
        # set var coord
        i = 0
        for var in self.variables: 
            var.coord = np.arange(i, i+var.dim)
            i += var.dim
        self.dim = i

        # set param coord
        i = 0
        for param in self.parameters: 
            param.coord = np.arange(i, i+param.dim)
            i += param.dim
        self.pdim = i

        # set constraint coordinates
        i = 0
        for constr in self.ordered_constr_list:
            constr.coord = np.arange(i, i+constr.dim)
            i += constr.dim
        self.cdim = i

    def set_constr_fn_dict(self, fn_dict:Dict[str,Callable], ignore:List[str]):
        required_constr_types = set(constr.constr_type for constr in self.constraints)
        success = True
        for constr_type in required_constr_types:
            try:        
                if constr_type in ignore:
                    print(f"Function ignored: {constr_type}")    
                else:
                    self.fn_dict[constr_type] = fn_dict[constr_type]
            except:
                print(f"No matched function: {constr_type}")
                success = False
        if success: 
            print("All constraint functions are set")
            self.set_coordinates()
            print("All Coordinates are set accordingly")

    def to_var_dict(self, x):
        return {var.name:x[var.coord] 
                for var in self.variables}
    def to_var_vec(self, x_dict):
        return np.hstack([x_dict[var.name] for var in self.variables])
    
    def to_param_dict(self, param):
        return {p.name:param[p.coord] 
                for p in self.parameters}
    def to_param_vec(self, param_dict):
        return np.hstack([param_dict[p.name] for p in self.parameters])
    
    def to_constr_dict(self, constr_val):
        return {f"{constr.constr_type}_{constr.name}":constr_val[constr.coord] 
                for constr in self.ordered_constr_list}
    
    def get_initial_value(self, init_dict={}, is_random=False):
        x0 = []
        for var in self.variables:
            if var.name in init_dict:
                value = init_dict[var.name]
            elif isinstance(var, Config):
                value = SubProblem.home_configs[var.robot]
            elif is_random:
                value = np.random.uniform(var.lb, var.ub)
            else:
                value = np.zeros(var.dim)
            x0.append(value)
        return np.hstack(x0)
            
    def get_objective_fn(self):
        trajectory = {robot:[] for robot in self.robots}
        for i in range(len(self.action_seq)):
            #add keyframe config
            for robot, config in self.keyframes[i].configs.items():
                trajectory[robot].append(config)
            #add via configs during action
            for robot, configs in self.action_seq[i].configs.items():
                for config in configs:
                    trajectory[robot].append(config)
        for robot, config in self.keyframes[-1].configs.items():
            trajectory[robot].append(config)

        if self.is_relaxed:
            objective = lambda param, x: 0.
        else:
            def objective(param, x):
                param_values = {p.name:param[p.coord] for p in self.parameters}
                var_values = {var.name:x[var.coord] for var in self.variables}
                instance_values = {**param_values, **var_values}
                
                obj_value = 0
                for robot in self.robots:
                    qs = []
                    for config in trajectory[robot]:
                        qs.append(instance_values[config.name])
                    qs = jnp.vstack(qs)
                    obj_value += jnp.sum((qs[1:] - qs[:-1])**2)
                return obj_value
        return objective 
    
    def get_constraint_fn(self):
        def constraints(param, x):
            param_values = {p.name:param[p.coord] for p in self.parameters}
            var_values = {var.name:x[var.coord] for var in self.variables}
            instance_values = {**param_values, **var_values}

            values = []
            for constr_name, constrs in self.constr_dict.items():
                constr_fn = self.fn_dict[constr_name]
                num_inputs = constrs[0].num_inputs
                input_lists = [[] for _ in range(num_inputs)]
                for constr in constrs:
                    for i, inp in enumerate(constr.inputs):
                        input_lists[i].append(instance_values[inp.name])
                inputs = [jnp.vstack(batch_input) for batch_input in input_lists]
                cval = jax.vmap(constr_fn, in_axes=[0]*num_inputs)(*inputs)
                values.append(cval.flatten())
            return jnp.hstack(values)
        return constraints


    #TODO:
    def get_jacobian_structure_indices(self):
        jac_structure = np.zeros((self.cdim, self.dim))
        for constr in self.ordered_constr_list:
            if constr.constr_type == "constant":
                x_coord = constr.inputs[0].coord
                jac_structure[constr.coord, x_coord] = 1
                continue
            
            var_coords = [instance.coord for instance in constr.inputs if instance.is_var]
            if len(var_coords) == 0: continue
            x_coords = np.hstack(var_coords)
            rowmat, colmat = np.meshgrid(constr.coord, x_coords, indexing='ij')
            jac_structure[rowmat.flatten(), colmat.flatten()] = 1
        row, col = np.nonzero(jac_structure)
        return row, col
    
    def get_jacobian_structure_fn(self):
        jac_struct = self.get_jacobian_structure_indices()
        jacobian_structure_fn = lambda : jac_struct
        return jacobian_structure_fn


In [3]:
# IPOPT Accelerator
import cyipopt
class IPOPTAcc:
    def __init__(
            self, builder:ProblemBuilder, 
            compile=False, sparse=False):
        self.builder = builder
        self.obj_fn = self.builder.get_objective_fn()
        self.constr_fn = self.builder.get_constraint_fn()
        self.grads = {}
        self.jacs = {}
        self.vg_obj_fn = jax.value_and_grad(self.obj_fn, argnums=1)
        self.vj_constr_fn = value_and_jacfwd(self.constr_fn, argnums=1)
        self.jac_struct = None #self.builder.get_jacobian_structure_indices() # (row, col)
        if sparse:
            # TODO: set jacobian sparsity indices
            self.jacobianstructure = lambda : self.jac_struct
        
        self.param = None
        self.debug_fn = None
        self.x_history = []
        self.iteration_ends = False
        if compile:
            vdim, pdim = self.builder.vdim, self.builder.pdim
            self.vg_obj_fn = jax.jit(self.vg_obj_fn).lower(np.zeros(pdim), np.zeros(vdim)).compile()
            self.vj_constr_fn = jax.jit(self.vj_constr_fn).lower(np.zeros(pdim), np.zeros(vdim)).compile()
    
    def set_debug_fn(self, debug_fn):
        self.debug_fn = debug_fn
    
    def set_problem(self, prob_dict):
        self.param = jnp.array(self.builder.to_param_vec(prob_dict))
    
    def check_constraint(self, x):
        cval = self.constr_fn(self.param, x)
        check_vector = (self.builder.clb < cval) & (cval < self.builder.cub)
        print("cval: ", self.builder.to_constr_dict(np.array(cval)))
        print("viol: ", self.builder.to_constr_dict(np.array(check_vector)))

    def hash(self, x):
        return hash(x.tobytes())

    def objective(self, x):
        value, grad = self.vg_obj_fn(self.param, x)
        key = self.hash(x)
        self.grads[key] = grad
        if (self.debug_fn is not None) and (self.iteration_ends):
            self.check_constraint(x)
            self.debug_fn(x)
            self.x_history.append(x)
            self.iteration_ends = False
        return value
    
    def gradient(self, x):
        key = self.hash(x)
        if key in self.grads:
            return self.grads[key]
        _, grad = self.vg_obj_fn(self.param, x)
        return grad
    
    def constraints(self, x):
        value, jac = self.vj_constr_fn(self.param, x)
        key = self.hash(x)
        self.jacs[key] = jac
        return value
    
    def jacobian(self, x):
        key = self.hash(x)
        if key not in self.jacs:
            _, jac = self.vj_constr_fn(self.param, x)
        else: jac = self.jacs[key]
        
        if self.jac_struct is not None:
            return jac[self.jac_struct]
        return jac

    def intermediate(self, alg_mod, iter_count, obj_value, inf_pr, inf_du, mu,
                     d_norm, regularization_size, alpha_du, alpha_pr,
                     ls_trials):
        self.iteration_ends = True

    def get_ipopt_solver(self):
        assert self.param is not None, "parameter is not set"
        self.x_history = [] # reset history

        ipopt = cyipopt.Problem(
            n=self.builder.dim, m=self.builder.cdim, problem_obj=self,
            lb=self.builder.xlb, ub=self.builder.xub, 
            cl=self.builder.clb, cu=self.builder.cub
        )
        ipopt.add_option("acceptable_iter", 2)
        ipopt.add_option("acceptable_tol", np.inf) #release
        ipopt.add_option("acceptable_dual_inf_tol", np.inf) #release
        ipopt.add_option("acceptable_constr_viol_tol", 1.)
        ipopt.add_option('mu_strategy', 'adaptive')
        ipopt.add_option('max_iter', 100)
        return ipopt

## Make Env

In [4]:
world = SDFWorld()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/


In [5]:
# refer to PREDEFINED_ROBOTS
gen3 = get_predefined_robot(world.vis, "gen3+hand_e")
hande = Gripper(world.vis, "hande", gen3.gripper.model, 
                gen3.gripper.tool_pose_offset, 
                max_width=0.05, scale=0.001, alpha=0.5, color='white')

In [6]:
def SE3_trans(xyz):
    return SE3.from_translation(jnp.array(xyz))

obj_mesh = "./sdf_world/assets/waffle_box/waffle_box_centered.obj"
ground = Box(world.vis, "ground", [1,1.8,0.5], 'gray', 0.3)
obj0 = Mesh(world.vis, "obj0", obj_mesh, color="green", alpha=0.3)
objg = Mesh(world.vis, "objg", obj_mesh, color="red", alpha=0.3)

In [7]:
def object_placement(obj, upright_axis, xy, yaw=None):
    if yaw is None:
        yaw = np.random.uniform(-np.pi, np.pi)
    obj_extents = obj.mesh.bounding_box.primitive.extents
    if upright_axis == "z":
        rot = SO3.from_z_radians(yaw)
        z_offset = obj_extents[-1]/2
    if upright_axis == "x":
        rot = SO3.from_z_radians(yaw)@SO3.from_y_radians(-np.pi/2)
        z_offset = obj_extents[0]/2
    elif upright_axis == "y":
        rot = SO3.from_z_radians(yaw)@SO3.from_x_radians(np.pi/2)
        z_offset = obj_extents[1]/2
    xyz = [*xy, z_offset]
    pose = SE3.from_rotation_and_translation(rot, jnp.array(xyz))
    return pose

gen3_home_config = np.array([0,0,0,np.pi/2, 0,np.pi/2, 0])
def get_problem():
    # xy, upright_axis= ground
    pose0 = object_placement(obj0, "x", xy=[0.4,0.3])
    poseg = object_placement(obj0, "z", xy=[0.4,-0.3])
    config_home = gen3_home_config
    return {"pose_init": pose0.parameters(),
            "pose_goal": poseg.parameters(),
            "config_home": config_home}

def visualize_problem(prob_dict):
    obj0.set_pose(SE3(prob_dict["pose_init"]))
    objg.set_pose(SE3(prob_dict["pose_goal"]))
    ground.set_pose(SE3_trans([0.4, 0, -0.25]))
    gen3.set_joint_angles(gen3_home_config)

In [8]:
prob_dict = get_problem()
visualize_problem(prob_dict)

### functions

In [9]:
#functions
#load learned functions
hande_pc = hande.get_hand_pc_wrt_tool_pose()
grasp_fn, grasp_reconst, grasp_embedding = load_grasp_fn("model/grasp_net_waffle")
inv_manip_fn_gen3 = load_manip_fn("model/manip_net_gen3", SE3.identity())
env = SDFContainer([ground]) #we can add more obstacles
robot_pc_fn = gen3.get_robot_pc_fn()
obj_pc = farthest_point_sampling(obj0.mesh.sample(100), 10)
fk_ee_fn = gen3.get_fk_ee_fn()

def R_grasp(grasp:Array, obj_pose:Array):
    grasp_pose = grasp_reconst(grasp)
    tool_pose = SE3(obj_pose).normalize() @ grasp_pose
    return tool_pose.parameters() # @ grasp_pose @ SE3_trans([0,0,-0.1])

def P_grasp(g):
    return grasp_fn(g)[0]

def M_inv(grasp:Array, obj_pose:Array):
    tool_pose = R_grasp(grasp, obj_pose)
    zflip_SE3 = SE3.from_rotation(SO3.from_z_radians(jnp.pi))
    #pre
    tool_pose = SE3(tool_pose)
    tool_pose_flip = (tool_pose @ zflip_SE3)
    candidates = jnp.vstack([pose.parameters() for pose in [tool_pose, tool_pose_flip]])
    return jax.vmap(inv_manip_fn_gen3)(candidates).max()

def K_robot(grasp:Array, obj_pose:Array, q:Array):
    tool_pose = R_grasp(grasp, obj_pose)
    tool_pose = SE3(tool_pose)
    ee_pose = SE3(fk_ee_fn(q))
    pos_err = tool_pose.translation() - ee_pose.translation()
    rot_err = (ee_pose.rotation().inverse() @ tool_pose.rotation()).log()
    return jnp.hstack([pos_err, 0.5*rot_err])

def D_hand(grasp, obj_pose):
    tool_pose = R_grasp(grasp, obj_pose)
    assigned_pc = jax.vmap(SE3(tool_pose).apply)(hande_pc)
    return env.distances(assigned_pc).min()

def D_robot(q:Array):
    assigned_pc = robot_pc_fn(q)
    return env.distances(assigned_pc).min()

def D_obj(q:Array, grasp:Array):
    ee_pose = SE3(fk_ee_fn(q))
    grasp_pose = grasp_reconst(grasp)
    obj_pose = ee_pose @ grasp_pose.inverse()
    pc = jax.vmap(obj_pose.apply)(obj_pc)
    return env.distances(pc).min()


hidden_dim:32, out_dim:7


In [10]:
#single arm reorientation
robots = ["gen3"]
home_configs = {
    "gen3":gen3_home_config,
}

obj_init_pose = Pose(name="init", is_var=False, value=prob_dict["pose_init"])
obj_goal_pose = Pose(name="goal", is_var=False, value=prob_dict["pose_goal"])
robot_home_configs = {
    "gen3":Config("home", "gen3", False, home_configs["gen3"])}

action_seq = [MoveFree("gen3"), MoveHold("gen3"), MoveFree("gen3")]
init_scene = Home(0, obj_init_pose, robot_home_configs)
goal_scene = Home(len(action_seq), obj_goal_pose, robot_home_configs)

In [11]:
fn_dict = {
    "grasp_prob": P_grasp,
    "inv_manipulability": M_inv,
    "hand_col_distance": D_hand,
    "kinematics_error": K_robot,
    "robot_col_distance": D_robot,
}
relaxed_prob = ProblemBuilder(robots, home_configs, is_relaxed=True)
relaxed_prob.build_problem(action_seq, init_scene, goal_scene)
relaxed_prob.set_constr_fn_dict(fn_dict, ignore=[])

set action_seq and keyframes
set variables and parameters
set constraints
All constraint functions are set
All Coordinates are set accordingly


In [12]:
pc_red = PointCloud(world.vis, "pc_red", np.zeros((10,3)), 0.01, "red")
pc_blue = PointCloud(world.vis, "pc_blue", np.zeros((10,3)), 0.01, "blue")
pc_green = PointCloud(world.vis, "pc_green", np.zeros((10,3)), 0.01, "green")

In [25]:
import cvxpy as cp

In [262]:
q = np.random.uniform(gen3.arm.model.lb, gen3.arm.model.ub)
ee_point = fk_ee_fn(q)[-3:]
gen3.set_joint_angles(q)

In [263]:
ee_point_g = grasp_embedding(SE3(prob_dict['pose_init']).inverse().apply(ee_point))
frame.set_pose(SE3(R_grasp(ee_point_g, prob_dict['pose_init'])))
obj_fn = lambda grasp: jnp.sum((grasp - ee_point_g)**2)
grad_fn = jax.grad(obj_fn)

In [323]:
def q_to_grasp(q):
    ee_point = fk_ee_fn(q)[-3:]
    ee_point_g = grasp_embedding(SE3(prob_dict['pose_init']).inverse().apply(ee_point))
    return ee_point_g

In [116]:
frame = Frame(world.vis, "frame")

In [250]:
def P_grasp_wrapper(grasp):
    return P_grasp(grasp)/1000

In [327]:
class Prob:
    def __init__(self, obj_fn):
        self.obj_fn = obj_fn
    def objective(self, x):
        frame.set_pose(SE3(R_grasp(x, prob_dict['pose_init'])))
        print(P_grasp_wrapper(x))
        return self.obj_fn(x)

prob = Prob(lambda x:0.)
# setattr(prob, "objective", obj_fn)
setattr(prob, "gradient", lambda x:np.zeros(3))
setattr(prob, "constraints", P_grasp_wrapper)
setattr(prob, "jacobian", jax.jacfwd(P_grasp_wrapper))

ipopt = cyipopt.Problem(
    n=3, m=1,problem_obj=prob,
    lb=-jnp.ones(3), ub=jnp.ones(3),
    cl=jnp.zeros(1), cu=jnp.full(1, np.inf)
)

ipopt.add_option("nlp_scaling_method", "none") #release
ipopt.add_option("acceptable_tol", 100.) #release
ipopt.add_option("acceptable_obj_change_tol", 0.1)
ipopt.add_option("acceptable_constr_viol_tol", 0.1) #release
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option('mu_strategy', 'adaptive')
ipopt.add_option('max_iter', 100)
sol, info = ipopt.solve(jnp.array([-0.4,0.6,0.5]))


This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

-0.13468066
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.35e-01 1.92e-01   0.0 0.00e+00    -  0.00e+

In [334]:
grasp = q_to_grasp(gen3_home_config)

In [335]:
#grasp = np.zeros(3)
class Prob:
    def __init__(self, obj_fn):
        self.obj_fn = obj_fn
    def objective(self, x):
        frame.set_pose(SE3(R_grasp(x, prob_dict['pose_init'])))
        print(P_grasp_wrapper(x))
        return self.obj_fn(x)
prob = Prob(lambda x:0.)
# setattr(prob, "objective", obj_fn)
setattr(prob, "gradient", lambda x:np.zeros(3))
setattr(prob, "constraints", P_grasp_wrapper)
setattr(prob, "jacobian", jax.jacfwd(P_grasp_wrapper))

ipopt = cyipopt.Problem(
    n=3, m=1,problem_obj=prob,
    lb=-jnp.ones(3), ub=jnp.ones(3),
    cl=jnp.zeros(1), cu=jnp.full(1, np.inf)
)
ipopt.add_option("nlp_scaling_method", "none") #release
ipopt.add_option("acceptable_tol", 100.) #release
ipopt.add_option("acceptable_obj_change_tol", 0.1)
ipopt.add_option("acceptable_constr_viol_tol", 0.1) #release
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option('mu_strategy', 'adaptive')
ipopt.add_option('max_iter', 100)
grasp, info = ipopt.solve(jnp.array(grasp))

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

-0.41118076
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 4.11e-01 3.77e-01   0.0 0.00e+00    -  0.00e+

In [ ]:
def farthest_point_sampling(points, num_samples):
    farthest_points = np.zeros((num_samples, 3))
    farthest_points[0] = points[np.random.randint(len(points))]
    distances = np.full(points.shape[0], np.inf)
    for i in range(1, num_samples):
        distances = np.minimum(distances, np.linalg.norm(points - farthest_points[i - 1], axis=1))
        farthest_points[i] = points[np.argmax(distances)]
    return farthest_points

In [315]:
import time

In [336]:
def K_robot_wrapper(x):
    q = x
    error = K_robot(grasp, prob_dict['pose_init'], x)
    return error
class Prob:
    def __init__(self, obj_fn):
        self.obj_fn = obj_fn
    def objective(self, x):
        gen3.set_joint_angles(x)
        time.sleep(0.1)
        return self.obj_fn(x)
prob = Prob(lambda x:0.)
setattr(prob, "gradient", lambda x:np.zeros(7))
setattr(prob, "constraints", K_robot_wrapper)
setattr(prob, "jacobian", jax.jacfwd(K_robot_wrapper))

ipopt = cyipopt.Problem(
    n=7, m=6,problem_obj=prob,
    lb=gen3.arm.model.lb, ub=gen3.arm.model.ub,
    cl=np.zeros(6), cu=np.zeros(6)
)

ipopt.add_option("nlp_scaling_method", "none") #release
ipopt.add_option("acceptable_tol", 100.) #release
ipopt.add_option("acceptable_obj_change_tol", 0.1)
ipopt.add_option("acceptable_constr_viol_tol", 0.1) #release
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option('mu_strategy', 'adaptive')
ipopt.add_option('max_iter', 10)
q, info = ipopt.solve(gen3_home_config)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       42
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 3.20e-01 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [319]:
prob.gradient(q)

array([0., 0., 0.])

In [303]:
K_robot(grasp, prob_dict['pose_init'], gen3_home_config)

Array([ 0.09446406,  0.36338875, -0.32020906,  0.08145443, -0.7028912 ,
        0.07869895], dtype=float32)

In [302]:
grasp

array([0.19736004, 0.01021076, 0.47550631])

Array(-116.672874, dtype=float32)

In [13]:
prob_dict = get_problem()
visualize_problem(prob_dict)

In [14]:
import time
opt_if_r = IPOPTAcc(relaxed_prob, True)
opt_if_r.set_problem(prob_dict)
def debug_fn_r(x):
    grasp = x
    obj_poses = [prob_dict['pose_init'], 
                 prob_dict['pose_goal']]

    pcs = []
    for obj_pose in obj_poses:
        tool_pose = R_grasp(grasp, obj_pose)
        pc = jax.vmap(SE3(tool_pose).normalize().apply)(hande_pc)
        pcs.append(pc)
    pcs = jnp.vstack(pcs)
    pc_red.reload(points=np.array(pcs, dtype=float))
    time.sleep(0.1)
opt_if_r.set_debug_fn(debug_fn_r)
# P_grasp_wrapper = lambda param, x:P_grasp(x)
# opt_if.vg_obj_fn = jax.value_and_grad(P_grasp_wrapper, argnums=1)

In [15]:
opt_if_r.set_problem(prob_dict)
ipopt_r = opt_if_r.get_ipopt_solver()
ipopt_r.add_option("acceptable_iter", 2)
ipopt_r.add_option("acceptable_constr_viol_tol", 0.01)
# ipopt_r.add_option("acceptable_obj_change_tol", 0.01)

x0_r = relaxed_prob.get_initial_value(is_random=True)
sol_r, info = ipopt_r.solve(x0_r)

debug_fn_r(sol_r)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       15
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

In [16]:
prob = ProblemBuilder(robots, home_configs, num_via_points=1, is_relaxed=False)
prob.build_problem(action_seq, init_scene, goal_scene)
prob.set_constr_fn_dict(fn_dict, ignore=[])

set action_seq and keyframes
{'gen3': []}
{'gen3': []}
{'gen3': []}
set variables and parameters
set constraints
All constraint functions are set
All Coordinates are set accordingly


In [18]:
prob.variables

[Var:config_MoveFree_0_gen3_via0,
 Var:grasp_1,
 Var:config_MoveHold_1_gen3_via0,
 Var:config_MoveFree_2_gen3_via0,
 Var:config_pickplace_gen3_1,
 Var:config_pickplace_gen3_2]

In [25]:
def debug_fn(x):
    x_dict = prob.to_var_dict(x)
    via_config_names = [var.name for var in prob.variables if "via" in var.name]
    grasp = x_dict["grasp_1"]
    configs = [x_dict["config_pickplace_gen3_1"],
               x_dict["config_pickplace_gen3_2"]]
    via_configs = [x_dict[name] for name in via_config_names]
    obj_poses = [prob_dict['pose_init'], 
                 prob_dict['pose_goal']]
    
    pcs = []
    for obj_pose in obj_poses:
        tool_pose = R_grasp(grasp, obj_pose)
        pc = jax.vmap(SE3(tool_pose).normalize().apply)(hande_pc)
        pcs.append(pc)
    pcs = jnp.vstack(pcs)
    pc_red.reload(points=np.array(pcs, dtype=float))
    
    pcs = []
    for config in configs:
        pc = robot_pc_fn(config)
        pcs.append(pc)
    pcs = jnp.vstack(pcs)
    pc_blue.reload(points=np.array(pcs, dtype=float))

    pcs = []
    for q in via_configs:
        pc = robot_pc_fn(q)
        pcs.append(pc)
    pcs = jnp.vstack(pcs)
    pc_green.reload(points=np.array(pcs, dtype=float))
    
    time.sleep(0.1)

In [26]:
import time
opt_if = IPOPTAcc(prob, True)
opt_if.set_problem(prob_dict)
opt_if.set_debug_fn(debug_fn)

In [84]:
from sdf_world.ik import *
config = IKConfig(jax.tree_util.Partial(fk_ee_fn), SE3.identity(), 0.001, 0.1, 100)
ik_fn = get_ik_fn()
def ik(tool_pose):
    init_val = (gen3_home_config, SE3(tool_pose), 1., 0, config)
    result = ik_fn(init_val=init_val)
    if result[3] == 21:
        print("failed")
    return result[0]

In [70]:
debug_fn_r(sol_r)

In [69]:
opt_if_r.check_constraint(sol_r)

cval:  {'inv_manipulability_rpickplace_gen3_1': array([0.7984695], dtype=float32), 'inv_manipulability_rpickplace_gen3_2': array([0.32209492], dtype=float32), 'grasp_prob_movehold_1_gen3': array([3.7278965], dtype=float32), 'hand_col_distance_rpickplace_gen3_1': array([0.12258333], dtype=float32), 'hand_col_distance_rpickplace_gen3_2': array([0.05470487], dtype=float32)}
viol:  {'inv_manipulability_rpickplace_gen3_1': array([ True]), 'inv_manipulability_rpickplace_gen3_2': array([ True]), 'grasp_prob_movehold_1_gen3': array([ True]), 'hand_col_distance_rpickplace_gen3_1': array([ True]), 'hand_col_distance_rpickplace_gen3_2': array([ True])}


In [93]:
ik_targets = [
    R_grasp(instance_dict["grasp_1"], instance_dict["pose_init"]),
    R_grasp(instance_dict["grasp_1"], instance_dict["pose_goal"]),
]

In [115]:
keyframe_qs = [ik(ik_target) for ik_target in ik_targets]

In [122]:
qs = {
    "config_MoveFree_0_gen3_via0":np.average([gen3_home_config, keyframe_qs[0]], axis=0),
    "config_pickplace_gen3_1":keyframe_qs[0],
    "config_MoveHold_1_gen3_via0":np.average([keyframe_qs[1], keyframe_qs[1]], axis=0),
    "config_pickplace_gen3_2":keyframe_qs[1],
    "config_MoveFree_2_gen3_via0":np.average([keyframe_qs[1], gen3_home_config], axis=0),
}

In [127]:
init_dict = {**relaxed_prob.to_var_dict(sol_r),
             **qs}
x0 = prob.get_initial_value(init_dict)
debug_fn(x0)

In [128]:
opt_if.set_problem(prob_dict)
ipopt = opt_if.get_ipopt_solver()
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option("acceptable_constr_viol_tol", 0.01)
# ipopt_r.add_option("acceptable_obj_change_tol", 0.01)

sol, info = ipopt.solve(x0)

debug_fn(sol)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:      456
Number of nonzeros in inequality constraint Jacobian.:      228
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       38
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       38
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

cval:  {'robot_col_distance_MoveFree_0_gen3_via0': array([0.15880519], dtype=float32), 'robot_col_distance_MoveHold_1_gen3_via0': array([0.05979121], dtype=fl

In [404]:
q = gen3_home_config
grasp = jnp.zeros(3)

In [148]:
def obj_fn(q, grasp):
    ee_pose = fk_ee_fn(q)
    tool_pose = R_grasp(grasp, prob_dict["pose_init"])    
    distance_vec = tool_pose[-3:] - ee_pose[-3:]
    return jnp.sum(distance_vec**2)

In [977]:
ee_point = fk_ee_fn(q)[-3:]

In [979]:
grasp_point = np.zeros(3)

In [ ]:
def prob_at_grasp_point(grasp):
    

In [660]:
def constr_fn(grasp):
    return -jnp.log(-P_grasp(grasp))

In [661]:
def unconstrained_obj(q, grasp, t):
    return t*obj_fn(q, grasp) - constr_fn(grasp)

In [662]:
unconstrained_obj(q, grasp, 1)

Array(6.878453, dtype=float32)

In [964]:
grad_fn = jax.grad(unconstrained_obj, argnums=1)
d = - grad_fn(q, grasp, 100)

#if np.linalg.norm(d) < 10:
grasp = grasp + d * 0.01 / np.linalg.norm(d)
print(d)

grasp_vis(grasp, prob_dict["pose_init"])

[-11.318775   -85.73246     -0.98866487]


In [973]:
grasp = np.zeros(3)

In [971]:
#grasp solving using sqp

In [976]:
jax.grad(P_grasp)(grasp)

Array([  5.486557, 120.04222 ,  80.05906 ], dtype=float32)

In [966]:
import cvxpy as cp

In [969]:
import cvxpy as cp
m,n = 1,3
# A = np.random.randn(m,n)
# b = np.random.randn(m)
# x = cp.Variable(n)
# lam = 0.1

f = 0. # cp.sum_squares(A@x - b) + lam*cp.norm1(x)
cons = [x >= 0]
cp.Problem(cp.Minimize(f), cons).solve(verbose=False, eps_abs=1e-8, eps_rel=1e-8)

0.03264679253764401

In [968]:
jax.grad(P_grasp)

Array([ 0.84203595,  0.00336818, -0.16181575], dtype=float32)

In [403]:
obj_fn(q, grasp)

Array(0.02530418, dtype=float32)

In [433]:
unconstrained_obj(q, grasp, 1)

Array(-6.534566, dtype=float32)

In [956]:
P_grasp(grasp)

Array(0.9343756, dtype=float32)

In [955]:
constr_fn(grasp)

Array(nan, dtype=float32)

In [402]:
obj_fn

<function __main__.obj_fn(q, grasp)>

In [159]:
def grasp_vis(grasp, obj_pose):
    tool_pose = R_grasp(grasp, obj_pose)
    pcs = jax.vmap(SE3(tool_pose).apply)(hande_pc)
    pc_red.reload(points=np.array(pcs, dtype=float))

In [161]:
gen3.set_joint_angles(q)
grasp_vis(grasp, prob_dict["pose_init"])

In [156]:
R_grasp(grasp, prob_dict["pose_init"])

Array([ 0.1284881 ,  0.29880372,  0.07257996, -0.94283575,  0.4       ,
        0.3       ,  0.0665    ], dtype=float32)

In [144]:
tool_pose[-3:] - ee_pose[-3:] #objective

Array([-0.02928689, -0.01278105, -0.0667685 ], dtype=float32)

In [130]:
opt_if.check_constraint(sol)

cval:  {'robot_col_distance_MoveFree_0_gen3_via0': array([0.15880519], dtype=float32), 'robot_col_distance_MoveHold_1_gen3_via0': array([0.06449807], dtype=float32), 'robot_col_distance_MoveFree_2_gen3_via0': array([0.15880519], dtype=float32), 'robot_col_distance_pickplace_gen3_1': array([0.12519789], dtype=float32), 'robot_col_distance_pickplace_gen3_2': array([0.05005348], dtype=float32), 'kinematics_error_pickplace_gen3_1': array([-1.0907650e-05,  9.7453594e-06, -1.3694167e-05,  1.9207214e-04,
       -9.5971435e-04,  1.0289898e-04], dtype=float32), 'kinematics_error_pickplace_gen3_2': array([ 5.7488680e-05, -6.4074993e-07, -4.5597553e-05,  3.1363950e-04,
       -9.7249402e-04,  1.2700254e-04], dtype=float32), 'grasp_prob_movehold_1_gen3': array([-1.0676041], dtype=float32)}
viol:  {'robot_col_distance_MoveFree_0_gen3_via0': array([ True]), 'robot_col_distance_MoveHold_1_gen3_via0': array([ True]), 'robot_col_distance_MoveFree_2_gen3_via0': array([ True]), 'robot_col_distance_pickpl

In [113]:
i = 0

In [151]:
print(i)
x = opt_if.x_history[i]
opt_if.debug_fn(x)
opt_if.check_constraint(x)
i += 1

35
cval:  {'hand_col_distance_rpickplace_gen3_1': array([0.00727862], dtype=float32), 'hand_col_distance_rpickplace_gen3_2': array([0.17604774], dtype=float32), 'grasp_prob_movehold_1_gen3': array([-282.00806], dtype=float32), 'inv_manipulability_rpickplace_gen3_1': array([0.30088285], dtype=float32), 'inv_manipulability_rpickplace_gen3_2': array([0.8671745], dtype=float32)}
viol:  {'hand_col_distance_rpickplace_gen3_1': array([False]), 'hand_col_distance_rpickplace_gen3_2': array([ True]), 'grasp_prob_movehold_1_gen3': array([False]), 'inv_manipulability_rpickplace_gen3_1': array([ True]), 'inv_manipulability_rpickplace_gen3_2': array([ True])}


In [117]:
P_grasp(x)

Array(-102.51603, dtype=float32)

In [90]:
[constr for constr in relaxed_prob.ordered_constr_list]

[Constr:_hand_col_distance_rpickplace_gen3_1,
 Constr:_hand_col_distance_rpickplace_gen3_2,
 Constr:_grasp_prob_movehold_1_gen3,
 Constr:_inv_manipulability_rpickplace_gen3_1,
 Constr:_inv_manipulability_rpickplace_gen3_2]

In [51]:
constr_fn = relaxed_prob.get_constraint_fn()

In [74]:
opt_if.check_constraint(sol)

{'hand_col_distance_rpickplace_gen3_1': Array([ True], dtype=bool),
 'hand_col_distance_rpickplace_gen3_2': Array([False], dtype=bool),
 'grasp_prob_movehold_1_gen3': Array([False], dtype=bool),
 'inv_manipulability_rpickplace_gen3_1': Array([False], dtype=bool),
 'inv_manipulability_rpickplace_gen3_2': Array([False], dtype=bool)}

In [54]:
cval = constr_fn(relaxed_prob.to_param_vec(prob_dict), sol)

In [56]:
relaxed_prob < cval

TypeError: '<' not supported between instances of 'ProblemBuilder' and 'ArrayImpl'

In [ ]:
ipopt_r.solve()

In [ ]:
solver_r = ipopt_r.get_ipopt_solver()

In [31]:
prob = ProblemBuilder(robots, home_configs, is_relaxed=False)
prob.build_problem(action_seq, init_scene, goal_scene)
prob.set_constr_fn_dict(fn_dict)

set action_seq and keyframes
set variables and parameters
set constraints
All constraint functions are set
Coordinates are set


In [352]:
obj_fn = prob.get_objective_fn()
constr_fn = prob.get_constraint_fn()

In [185]:
obj_fn = relaxed_prob.get_objective_fn()
constr_fn = relaxed_prob.get_constraint_fn()
jac_struct_fn = relaxed_prob.get_jacobian_structure_fn()
x0 = relaxed_prob.get_initial_value(prob_dict, is_random=False)

In [186]:
pc_vis = PointCloud(world.vis, "pc", np.zeros((10,3)), 0.01, "red")
pc_vis_b = PointCloud(world.vis, "pc", np.zeros((10,3)), 0.01, "blue")

In [137]:
del pc_vis, pc_vis_b

In [187]:
import time
def debug_fn_r(x):
    x_dict = relaxed_prob.to_var_dict(x)

    grasp = x_dict["grasp_1"]
    obj_pose0 = x_dict["pose_init"]
    obj_poseg = x_dict["pose_goal"]
    obj_poses = [obj_pose0, obj_poseg]

    pcs = []
    for obj_pose in obj_poses:
        tool_pose = R_grasp(grasp, obj_pose)
        pc = jax.vmap(SE3(tool_pose).normalize().apply)(hande_pc)
        pcs.append(pc)
    pcs = jnp.vstack(pcs)
    
    pc_vis.reload(points=np.array(pcs, dtype=float))
    time.sleep(0.1)

ipopt_r = IPOPTAcc(relaxed_prob, True, debug_fn=debug_fn_r)
solver_r = ipopt_r.get_ipopt_solver()

In [188]:
sol_dict = relaxed_prob.to_var_dict(sol)
grasp = sol_dict["grasp_1"]
pose_init = sol_dict["pose_init"]
pose_goal = sol_dict["pose_goal"]

In [171]:
SE3(pose_init)

SE3(wxyz=[ 0.67109996 -0.22284    -0.5        -0.22284   ], xyz=[0.40034    0.30023998 0.06655   ])

In [165]:
debug_fn_r(sol)

In [115]:
relaxed_prob.to_var_dict(sol)

{'grasp_1': array([ 7.72707770e-01, -1.71476841e-04,  4.06365361e-01]),
 'pose_init': array([ 0.70657744,  0.02735457, -0.50000001,  0.02735457,  0.40000001,
         0.30000003,  0.0665    ]),
 'pose_goal': array([ 2.94549686e-01,  0.00000000e+00, -1.62890279e-13, -9.55636107e-01,
         4.00000014e-01, -3.00000027e-01,  3.50000007e-02])}

In [189]:
prob_dict = get_problem()
visualize_problem(prob_dict)

In [190]:
x0 = relaxed_prob.get_initial_value(prob_dict)
debug_fn_r(x0)
sol, info = solver_r.solve(x0)

solver_r.add_option("acceptable_iter", 2)
solver_r.add_option("acceptable_tol", np.inf) #release
solver_r.add_option("acceptable_dual_inf_tol", np.inf) #release
solver_r.add_option("acceptable_obj_change_tol", 0.1)
solver_r.add_option("acceptable_constr_viol_tol", 2.)
solver_r.add_option('mu_strategy', 'adaptive')
solver_r.add_option('max_iter', 100)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       15
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       17
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       17
                     variables with only upper bounds:        0
Total number of equality constraints.................:       14
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.18e+02 1.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

KeyboardInterrupt: 

In [127]:
jac_struct_mat = np.zeros((relaxed_prob.cdim, relaxed_prob.dim))
jac_struct_mat[relaxed_prob.get_jacobian_structure_indices()] = 1

In [172]:
for line in jac_struct_mat:
    print(line)

[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [120]:
prob_dict["pose_init"]

Array([ 0.02366901,  0.7067106 , -0.02366901,  0.7067105 ,  0.4       ,
        0.3       ,  0.0665    ], dtype=float32)

In [121]:
[i.coord for i in relaxed_prob.instances]
[c.coord for c in relaxed_prob.ordered_constr_list]

[array([0]),
 array([1]),
 array([2]),
 array([3]),
 array([4]),
 array([ 5,  6,  7,  8,  9, 10, 11]),
 array([12, 13, 14, 15, 16, 17, 18])]

In [147]:
cval = constr_fn(sol)
check = (relaxed_prob.clb-0.02 < cval) & (cval < relaxed_prob.cub+0.02)
relaxed_prob.to_constr_dict(check)

{'hand_col_distance_rpickplace_gen3_1': Array([ True], dtype=bool),
 'hand_col_distance_rpickplace_gen3_2': Array([ True], dtype=bool),
 'grasp_prob_movehold_1_gen3': Array([ True], dtype=bool),
 'inv_manipulability_rpickplace_gen3_1': Array([ True], dtype=bool),
 'inv_manipulability_rpickplace_gen3_2': Array([ True], dtype=bool),
 'constant_pose_init': Array([ True,  True, False,  True,  True,  True,  True], dtype=bool),
 'constant_pose_goal': Array([ True,  True,  True,  True,  True,  True,  True], dtype=bool)}

In [23]:
constr_fn(x0)

Array([ 7.06577420e-01,  2.73545738e-02, -7.06577480e-01,  2.73545720e-02,
        4.00000006e-01,  3.00000012e-01,  6.65000007e-02,  2.94549674e-01,
        0.00000000e+00,  0.00000000e+00, -9.55636084e-01,  4.00000006e-01,
       -3.00000012e-01,  3.50000001e-02, -1.34505019e-01,  8.44511390e-02,
       -1.16672874e+02,  2.12667227e-01,  8.75632703e-01], dtype=float32)

In [15]:
import time
def debug_fn(x):
    x_dict = prob.to_var_dict(x)

    grasp = x_dict["grasp_1"]
    obj_pose0 = x_dict["pose_init"]
    obj_poseg = x_dict["pose_goal"]
    obj_poseg = x_dict["pose_goal"]
    config_pick = x_dict["config_pickplace_gen3_1"]
    config_place = x_dict["config_pickplace_gen3_2"]
    obj_poses = [obj_pose0, obj_poseg]
    configs = [config_pick, config_place]
    
    gen3.set_joint_angles(config_pick)
    pcs = []
    pcs_robot = []
    for obj_pose in obj_poses:
        tool_pose = R_grasp(grasp, obj_pose)
        pc = jax.vmap(SE3(tool_pose).apply)(hande_pc)
        pcs.append(pc)
    for config in configs:
        pc = robot_pc_fn(config)
        pcs_robot.append(pc)
    pcs = jnp.vstack(pcs)
    pcs_robot = jnp.vstack(pcs_robot)
    
    pc_vis.reload(points=np.array(pcs, dtype=float))
    pc_vis_b.reload(points=np.array(pcs_robot, dtype=float))
    time.sleep(0.1)

In [109]:
obj_fn(x0)

IndexError: arrays used as indices must be of integer (or boolean) type

In [107]:
solver_r.solve(x0)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       14
Number of nonzeros in inequality constraint Jacobian.:       15
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       17
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       17
                     variables with only upper bounds:        0
Total number of equality constraints.................:       14
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.18e+02 1.25e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

AssertionError: [None None]

In [ ]:
solver_r.solve()

In [ ]:
def debug_fn_r(x):
    x_dict = prob.to_var_dict(x)

    grasp = x_dict["grasp_1"]
    obj_pose0 = x_dict["pose_init"]
    obj_poseg = x_dict["pose_goal"]
    obj_poseg = x_dict["pose_goal"]
    config_pick = x_dict["config_pickplace_gen3_1"]
    config_place = x_dict["config_pickplace_gen3_2"]
    obj_poses = [obj_pose0, obj_poseg]
    configs = [config_pick, config_place]
    
    gen3.set_joint_angles(config_pick)
    pcs = []
    pcs_robot = []
    for obj_pose in obj_poses:
        tool_pose = R_grasp(grasp, obj_pose)
        pc = jax.vmap(SE3(tool_pose).apply)(hande_pc)
        pcs.append(pc)
    for config in configs:
        pc = robot_pc_fn(config)
        pcs_robot.append(pc)
    pcs = jnp.vstack(pcs)
    pcs_robot = jnp.vstack(pcs_robot)
    
    pc_vis.reload(points=np.array(pcs, dtype=float))
    pc_vis_b.reload(points=np.array(pcs_robot, dtype=float))
    time.sleep(0.1)

In [34]:
ipopt = cyipopt.Problem(
    n=prob.dim, m=prob.cdim, problem_obj=ipopt_prob,
    lb=prob.xlb, ub=prob.xub, cl=prob.clb, cu=prob.cub
)
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option("acceptable_tol", np.inf) #release
ipopt.add_option("acceptable_dual_inf_tol", np.inf) #release
ipopt.add_option("acceptable_constr_viol_tol", 1.)
ipopt.add_option('mu_strategy', 'adaptive')
ipopt.add_option('max_iter', 100)

In [35]:
prob_dict = get_problem()
visualize_problem(prob_dict)
x0 = prob.get_initial_value(prob_dict=prob_dict)
debug_fn(x0)

In [36]:
sol, info = ipopt.solve(x0)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:      141
Number of nonzeros in inequality constraint Jacobian.:       17
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       38
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       38
                     variables with only upper bounds:        0
Total number of equality constraints.................:       33
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.18e+02 1.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

KeyboardInterrupt: 

In [115]:
prob.to_constr_dict(constr_fn(sol))

{'constant_pose_init': Array([ 0.5322116 , -0.46554023, -0.4953336 , -0.4655402 ,  0.3999918 ,
         0.29999435,  0.06649885], dtype=float32),
 'constant_config_home': Array([3.5825551e-30, 4.4002900e-20, 3.2318808e-30, 1.5707927e+00,
        9.1483869e-30, 1.5707927e+00, 4.1402790e-20], dtype=float32),
 'constant_pose_goal': Array([ 9.9237537e-01,  0.0000000e+00,  1.1496057e-05, -2.3711840e-02,
         3.9999181e-01, -2.9999435e-01,  3.4999400e-02], dtype=float32),
 'robot_col_distance_pickplace_gen3_1': Array([0.06459785], dtype=float32),
 'robot_col_distance_pickplace_gen3_2': Array([-0.06604733], dtype=float32),
 'kinematics_error_pickplace_gen3_1': Array([ 0.00603589,  0.08153071, -0.0875651 ,  0.28978324,  0.7486243 ,
         0.36019212], dtype=float32),
 'kinematics_error_pickplace_gen3_2': Array([ 0.03084141, -0.15086997, -0.13048118,  0.0770038 ,  0.78534704,
         0.4328842 ], dtype=float32),
 'grasp_prob_movehold_1_gen3': Array([-155.91768], dtype=float32)}

In [124]:
cval = constr_fn(sol)

In [128]:
pose0.parameters()

Array([ 0.5322244 , -0.46555048, -0.5322245 , -0.46555045,  0.4       ,
        0.3       ,  0.0665    ], dtype=float32)

In [127]:
prob.to_var_dict(sol)

{'grasp_1': array([0.71341723, 0.01297885, 0.40685062]),
 'config_pickplace_gen3_1': array([-1.04212389,  1.50089116, -2.81963958, -0.01304467, -2.64904021,
         2.38936498, -0.52994212]),
 'config_pickplace_gen3_2': array([ 0.41450362,  1.4010367 ,  1.5985031 ,  0.03220662,  1.78315882,
        -2.1951264 , -1.07828692]),
 'pose_init': array([ 0.53222444, -0.46555049, -0.49999998, -0.46555046,  0.40000002,
         0.30000002,  0.0665    ]),
 'config_home': array([-1.87086086e-23, -2.58729271e-24,  5.56837089e-24,  1.57079630e+00,
        -2.71196162e-24,  1.57079630e+00,  6.41107047e-24]),
 'pose_goal': array([ 9.99718869e-01,  0.00000000e+00,  8.40803638e-13, -2.37122455e-02,
         4.00000018e-01, -3.00000022e-01,  3.49999999e-02])}

In [125]:
constr_check = (prob.clb-0.02 < cval) & (cval < prob.cub+0.02)
prob.to_constr_dict(constr_check)

{'constant_pose_init': Array([ True,  True, False,  True,  True,  True,  True], dtype=bool),
 'constant_config_home': Array([ True,  True,  True,  True,  True,  True,  True], dtype=bool),
 'constant_pose_goal': Array([ True,  True,  True,  True,  True,  True,  True], dtype=bool),
 'robot_col_distance_pickplace_gen3_1': Array([ True], dtype=bool),
 'robot_col_distance_pickplace_gen3_2': Array([ True], dtype=bool),
 'kinematics_error_pickplace_gen3_1': Array([ True, False,  True,  True,  True,  True], dtype=bool),
 'kinematics_error_pickplace_gen3_2': Array([ True,  True,  True,  True,  True,  True], dtype=bool),
 'grasp_prob_movehold_1_gen3': Array([ True], dtype=bool)}

In [74]:
constr_vals = prob.to_constr_dict(constr_fn(sol))
constr_lb = prob.to_constr_dict(constr_fn(sol))

for constr_name, constr_val in constr_vals.items():
    print(constr_name,
          constr_val)

{'constant': array([ 0.92122221,  0.        ,  0.        ,  0.38903674,  0.40000001,
        -0.30000001,  0.035     ]),
 'grasp_prob': array([inf]),
 'kinematics_error': array([0., 0., 0., 0., 0., 0.]),
 'robot_col_distance': array([inf])}

In [129]:
x_dict = prob.to_var_dict(sol)
x_dict

{'grasp_1': array([0.71341723, 0.01297885, 0.40685062]),
 'config_pickplace_gen3_1': array([-1.04212389,  1.50089116, -2.81963958, -0.01304467, -2.64904021,
         2.38936498, -0.52994212]),
 'config_pickplace_gen3_2': array([ 0.41450362,  1.4010367 ,  1.5985031 ,  0.03220662,  1.78315882,
        -2.1951264 , -1.07828692]),
 'pose_init': array([ 0.53222444, -0.46555049, -0.49999998, -0.46555046,  0.40000002,
         0.30000002,  0.0665    ]),
 'config_home': array([-1.87086086e-23, -2.58729271e-24,  5.56837089e-24,  1.57079630e+00,
        -2.71196162e-24,  1.57079630e+00,  6.41107047e-24]),
 'pose_goal': array([ 9.99718869e-01,  0.00000000e+00,  8.40803638e-13, -2.37122455e-02,
         4.00000018e-01, -3.00000022e-01,  3.49999999e-02])}

In [130]:
gen3.set_joint_angles(x_dict['config_pickplace_gen3_2'])

In [140]:
#dualarm setting
robots = ["gen3", "panda"]
home_configs = {
    "gen3":np.zeros(7),
    "panda":np.zeros(7),
}

prob = ProblemBuilder(robots, home_configs)

#dual arm reorientation
obj_init_pose = Pose(name="init", is_var=False, value=np.zeros(7))
obj_goal_pose = Pose(name="goal", is_var=False, value=np.zeros(7))
robot_home_configs = {
    "gen3":Config("gen3_home", "gen3", False, home_configs["gen3"]), 
    "panda":Config("panda_home", "panda", False, home_configs["panda"])}

action_seq = [MoveFree("gen3"), MoveHold("gen3"), MoveHold("panda"), MoveFree("panda")] #dual arm
init_scene = Home(0, obj_init_pose, robot_home_configs)
goal_scene = Home(len(action_seq), obj_goal_pose, robot_home_configs)

prob.build_problem(action_seq, init_scene, goal_scene)


set action_seq and keyframes
set variables and parameters
set constraints
